In [1]:
import hashlib
import socket
import time
import threading
import string    
import random

In [2]:
#Private
privateString = str("lfAtVRDVhEsIilyjCTKo")
print("privateString: " + str(privateString))
print("Encoded: ", privateString.encode())

result = hashlib.sha1(privateString.encode())
print("SHA1 Hashed: ", result)
print("Hexadecimal: ", result.hexdigest())
 
# Random
randomString = str(''.join(random.choices(string.ascii_letters, k=20)))
print("randomString: ", randomString)
print("Encoded: ", randomString.encode())

result2 = hashlib.sha1(randomString.encode())
print("SHA1 Hashed: ", result2)
print("Hexadecimal: ", result2.hexdigest())

# Concatenated
sha1Result = hashlib.sha1((randomString+privateString).encode())
print("SHA1 Hashed: ", sha1Result)
print("Hexadecimal: ", sha1Result.hexdigest())

privateString: lfAtVRDVhEsIilyjCTKo
Encoded:  b'lfAtVRDVhEsIilyjCTKo'
SHA1 Hashed:  <sha1 _hashlib.HASH object @ 0x000001AF32A2AE50>
Hexadecimal:  5ea54def00326916d15208ce14460b5d0e49632e
randomString:  ODhhexuTFRaiHDDtBarG
Encoded:  b'ODhhexuTFRaiHDDtBarG'
SHA1 Hashed:  <sha1 _hashlib.HASH object @ 0x000001AF32A2AAB0>
Hexadecimal:  4d7c403308dcd897756bb049ee4a2eb5f2f8a003
SHA1 Hashed:  <sha1 _hashlib.HASH object @ 0x000001AF32A2AED0>
Hexadecimal:  07159b584fca658d9d45fdbf7d4f47cd03ed1b72


In [3]:
class ServerSocket:
    def __init__(self, HOST, PORT):
        self.HOST = HOST
        self.PORT = PORT
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((HOST, PORT))

    def start(self):
        self.socket.listen()
        print(f'[LISTENING] {self.HOST}:{self.PORT}')

        while True:
            conn, addr = self.socket.accept()
            print(f'[NEW CONNECTION] {addr}')

            thread = threading.Thread(target=self.handle_connection, args=(conn, addr))
            thread.start()
            print(f'[ACTIVE THREADS] {threading.activeCount()-1}')


    def handle_connection(self, conn, addr):
        if self.auth(conn, addr):
            connected = True
            while connected:   
                self.sendPacket(conn, addr, 1)
                connected = self.recievePacket(conn, addr)

        conn.close()  


    def auth(self, conn, addr):
        startString = "Start_Connection"
        privateString = "PRIVATE_KEY_THIRTY_TWO_CHARACTER"
        randomString = str(''.join(random.choices(string.ascii_letters, k=32)))
        sha1Result = hashlib.sha1(privateString.encode() + randomString.encode()).hexdigest()

        start = conn.recv(len(startString)).decode()
        print(f'[AUTH RECIEVED] Start: {start}')

        if start == startString:
            conn.send(randomString.encode())
            print(f'[AUTH SENT] Random: {randomString}')

            result = conn.recv(40).decode()
            print(f'[AUTH RECIEVED] SHA1: {result}')

            if result == sha1Result:
                message = "Authentication succesful. Do you wish to proceed?"
                conn.send(message.encode())
                print(f'[AUTH SENT] Message: {message}')

                answer = conn.recv(1).decode()
                print(f'[AUTH RECIEVED] Answer: {answer}')

                if answer == "Y":
                    return True
            else:
                message = "Authentication unsuccesful."
                conn.send(message.encode())
                print(f'[AUTH SENT] Message: {message}')

        return False

    
    def sendPacket(self, conn, addr, packet_type):
        if packet_type == 0: #Quesiton
            payload = "Question?".encode()
            payload_size = len(payload)

        elif packet_type == 1: #Time
            payload = str(time.ctime()).encode()
            payload_size = len(payload)

        elif packet_type == 2: #End
            payload = "End".encode()
            payload_size = len(payload)

        header = bytes([packet_type, payload_size])
  
        conn.send(header)
        conn.send(payload)

        print(f'[SENT] Header: {header} - Payload: {payload}')


    def recievePacket(self, conn, addr):
        header = conn.recv(2)
        packet_type = header[0]
        payload = '-empty-'

        if packet_type == 3: #Guess
            payload_size = header[1]
            payload = conn.recv(payload_size).decode()

        print(f'[RECIEVED] Header: {header} - Payload: {payload}')

        if packet_type == 1:
            print(f'[DISCONNECTED] {addr}')
            return False
        else:
            return True


In [4]:
if __name__=="__main__":
    s = ServerSocket('127.0.0.1', 3001)
    s.start()


[LISTENING] 127.0.0.1:3001
[NEW CONNECTION] ('127.0.0.1', 58547)
[AUTH RECIEVED] Start: Start_Connection[ACTIVE THREADS] 6

[AUTH SENT] Random: NizOTDpIXPMKxePwulKwBgEzcbljfvCW
[AUTH RECIEVED] SHA1: 368f7ddfcdccbb10f4870870bfa609c7a8a552dc
[AUTH SENT] Message: Authentication succesful. Do you wish to proceed?
